<a href="https://colab.research.google.com/github/MusliHyseni/MNIST-Deep-Learning-classification/blob/main/ConvolutionalMNIST_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
import torchvision
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import requests
from pathlib import Path
import random

device = "cuda" if torch.cuda.is_available() else "cpu"
train_data = datasets.FashionMNIST(
    root = "data",
    train=True,
    download=True,
    transform=ToTensor(),
    target_transform=None
)

test_data = datasets.FashionMNIST(
    root = "data",
    train=False,
    download=True,
    transform=ToTensor(),
)

class_names = train_data.classes


class FashionMNISTConv(nn.Module):
    def __init__(self, input_shape: int, model_config: tuple):
        super().__init__()
        hidden_units, output_shape = model_config

        self.block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=1, padding=1)
        )

        self.block_2 = nn.Sequential(
            nn.Conv2d(hidden_units, hidden_units, 3, 1),
            nn.ReLU(),
            nn.Conv2d(hidden_units, hidden_units, 3, 1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        dummy_input = torch.randn((1, input_shape, 28, 28))
        dummy_output = torch.reshape(dummy_input, (dummy_input(0),-1))
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(dummy_output, output_shape)
        )


    def _get_conv_output(self, x):
        x = self.block_1(x)
        x = self.block_2(x)
        return x.view(x.size(0), -1)

    def forward(self, x: torch.Tensor):
        x = self.block_1(x)
        x = self.block_2(x)
        x = self.classifier(x)
        return x
torch.manual_seed(42)
model_0 = FashionMNISTConv(1, (10, len(class_names))).to(device)
print(model_0)

def predictions(model: torch.nn.Module, data: list, device = device):
    pred_probs = []
    model.eval()
    with torch.inference_mode():
        for sample in data:
            sample = torch.unsqueeze(sample, dim=0).to(device)
            pred_logit = model(sample)
            pred_prob = torch.softmax(pred_logit.unsqueeze(), dim=1)
            pred_probs.append(pred_prob.cpu())
    return torch.cat(pred_probs)

random.seed(42)
test_samples = []
test_labels = []
for sample, label in random.sample(list(test_data), 9 ):
    test_samples.append(sample)
    test_labels.append(label)

pred_probs = predictions(model_0, test_samples)
print(pred_probs[:2])

TypeError: ignored